In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import copy

In [2]:
import torch
torch.cuda.empty_cache()

In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'pseudo_cascade_swin_t'
# config file 들고오기
cfg = Config.fromfile('./_cascade_rcnn/cascade_swin_t_fpn_3x_coco.py')
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.dataset.classes = classes
cfg.data.train.dataset.img_prefix = root 
cfg.data.train.dataset.ann_file = './upsampling_stratified_kfold/cv_train_1.json' 


# cfg.data.val.dataset.classes = classes
# cfg.data.val.dataset.img_prefix = root 
# cfg.data.val.dataset.ann_file = './upsampling_stratified_kfold/cv_val_1.json' 

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = './upsampling_stratified_kfold/cv_val_1.json' 

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'

cfg.data.samples_per_gpu = 1

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + model_name

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)

In [9]:
# build_dataset
# datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.val)]
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...


Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7fef6add0200>
Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/tempfile.py", line 448, in __del__
    self.close()
  File "/opt/conda/envs/detection/lib/python3.7/tempfile.py", line 444, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpzp2gipqw/tmp_wq7h9yv.py'
Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7fef6add0200>
Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/tempfile.py", line 448, in __del__
    self.close()
  File "/opt/conda/envs/detection/lib/python3.7/tempfile.py", line 444, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmp35xf8g17/tmpapy53dbf.py'


Done (t=0.65s)
creating index...
index created!


In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [11]:
# 모델 학습
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-04-04 07:30:46,536 - mmdet - INFO - Start running, host: root@6f46492e33de, work_dir: /opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/psudo_cascade_swin_t
2022-04-04 07:30:46,537 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(LOW        

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


wandb: Currently logged in as: ysw2946 (use `wandb login --relogin` to force relogin)


2022-04-04 07:32:08,338 - mmdet - INFO - Epoch [1][50/8604]	lr: 4.995e-06, eta: 1 day, 18:53:15, time: 1.496, data_time: 0.065, memory: 19536, loss_rpn_cls: 0.6989, loss_rpn_bbox: 0.1217, s0.loss_cls: 1.2576, s0.acc: 76.7188, s0.loss_bbox: 0.0078, s1.loss_cls: 0.5857, s1.acc: 77.5586, s1.loss_bbox: 0.0011, s2.loss_cls: 0.3235, s2.acc: 72.1562, s2.loss_bbox: 0.0003, loss: 2.9967, grad_norm: 31.2999
2022-04-04 07:33:09,604 - mmdet - INFO - Epoch [1][100/8604]	lr: 9.990e-06, eta: 1 day, 14:59:12, time: 1.225, data_time: 0.011, memory: 19536, loss_rpn_cls: 0.5700, loss_rpn_bbox: 0.1306, s0.loss_cls: 0.5407, s0.acc: 91.0352, s0.loss_bbox: 0.1534, s1.loss_cls: 0.2002, s1.acc: 94.6367, s1.loss_bbox: 0.0471, s2.loss_cls: 0.0887, s2.acc: 95.8516, s2.loss_bbox: 0.0073, loss: 1.7380, grad_norm: 6.4252
2022-04-04 07:34:16,584 - mmdet - INFO - Epoch [1][150/8604]	lr: 1.499e-05, eta: 1 day, 14:45:59, time: 1.340, data_time: 0.015, memory: 19536, loss_rpn_cls: 0.4157, loss_rpn_bbox: 0.1320, s0.loss_c

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2120/2120, 3.6 task/s, elapsed: 593s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.35s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=11.16s).
Accumulating evaluation results...


2022-04-05 03:51:39,225 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.531
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.693
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.578
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.649
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.649
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.649
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.334
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.28s).


2022-04-05 03:51:39,488 - mmdet - INFO - The previous best checkpoint /opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/psudo_cascade_swin_t/best_bbox_mAP_epoch_6.pth was removed
2022-04-05 03:51:42,959 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_7.pth.
2022-04-05 03:51:42,960 - mmdet - INFO - Best bbox_mAP is 0.5310 at 7 epoch.
2022-04-05 03:51:42,978 - mmdet - INFO - Epoch(val) [7][2120]	bbox_mAP: 0.5310, bbox_mAP_50: 0.6930, bbox_mAP_75: 0.5780, bbox_mAP_s: 0.0320, bbox_mAP_m: 0.1870, bbox_mAP_l: 0.6110, bbox_mAP_copypaste: 0.531 0.693 0.578 0.032 0.187 0.611
2022-04-05 03:52:34,122 - mmdet - INFO - Epoch [8][50/8604]	lr: 5.500e-05, eta: 13:45:48, time: 1.022, data_time: 0.069, memory: 20773, loss_rpn_cls: 0.0314, loss_rpn_bbox: 0.0554, s0.loss_cls: 0.3369, s0.acc: 89.3281, s0.loss_bbox: 0.1627, s1.loss_cls: 0.1482, s1.acc: 90.6094, s1.loss_bbox: 0.1421, s2.loss_cls: 0.0710, s2.acc: 91.1398, s2.loss_bbox: 0.0795, loss: 1.0271, gr

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/psudo_cascade_swin_t/None.log.json'

In [ ]:
import torch
torch.cuda.empty_cache()